This notebook is for troubleshooting plastomes using NCBI-provided software pipelines. It was a suggestion from NCBI team rejected the annotation twice after uploading.

In [1]:
import os

# Defining variables for files
template = 'plastomes/final/template.sbt'
gb_cc = 'plastomes/final/Crepis_callicephala.gb'
fasta_cc = 'plastomes/in/crepis_callicephalac.fasta'

# command itself:
# table2asn -t template.sbt -i sequence.fsa

In [ ]:
# check for fasta

from Bio import SeqIO, SeqRecord

try:
    with open(fasta_cc, "r") as fasta_file:
        print(f"File {fasta_cc} does exist.")
except FileNotFoundError:
    print(f"File '{fasta_cc}' not found.\nConverting genbank file '{gb_cc}' to FASTA...")
    try:
        first_record = next(SeqIO.parse(gb_cc, "genbank"))
        SeqIO.write(first_record, fasta_cc, "fasta")
        print(f"Successfully created '{fasta_cc}'.")
    except FileNotFoundError:
        print(f"Error: The source GenBank file '{gb_cc}' was not found.")
    except StopIteration:
        print(f"Error: The GenBank file '{gb_cc}' is empty.")

File 'plastomes/in/crepis_callicephalac.fasta' not found.
Converting genbank file 'plastomes/final/Crepis_callicephala.gb' to FASTA...
Successfully created 'plastomes/in/crepis_callicephalac.fasta'.
